In [6]:
from jqdatasdk import *
auth('18950478833','Tonylian0814@') 

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改
auth success 


In [7]:
get_query_count()

{'total': 1000000, 'spare': 1000000}

In [8]:
df = get_bars('600519.XSHG', 5, unit='1d',fields=['date','open','high','low','close'],include_now=False,end_dt='2018-12-05')
print(df)

         date    open    high     low   close
0  2018-11-28  550.85  559.19  545.50  557.00
1  2018-11-29  566.03  569.80  549.01  551.60
2  2018-11-30  557.00  568.89  553.00  565.00
3  2018-12-03  589.00  605.00  584.77  601.20
4  2018-12-04  602.99  604.26  595.00  602.23


In [75]:
# 设置起止时间
start='2016-07-01'
end='2017-07-01'
# 设置调仓周期
periods=(5,10,20)

In [76]:
quantiles=5
#获取日期列表
date_list = get_trade_days(start_date=start,end_date=end,count=None)#获取回测日期间的所有交易日
date_list

array([datetime.date(2016, 7, 1), datetime.date(2016, 7, 4),
       datetime.date(2016, 7, 5), datetime.date(2016, 7, 6),
       datetime.date(2016, 7, 7), datetime.date(2016, 7, 8),
       datetime.date(2016, 7, 11), datetime.date(2016, 7, 12),
       datetime.date(2016, 7, 13), datetime.date(2016, 7, 14),
       datetime.date(2016, 7, 15), datetime.date(2016, 7, 18),
       datetime.date(2016, 7, 19), datetime.date(2016, 7, 20),
       datetime.date(2016, 7, 21), datetime.date(2016, 7, 22),
       datetime.date(2016, 7, 25), datetime.date(2016, 7, 26),
       datetime.date(2016, 7, 27), datetime.date(2016, 7, 28),
       datetime.date(2016, 7, 29), datetime.date(2016, 8, 1),
       datetime.date(2016, 8, 2), datetime.date(2016, 8, 3),
       datetime.date(2016, 8, 4), datetime.date(2016, 8, 5),
       datetime.date(2016, 8, 8), datetime.date(2016, 8, 9),
       datetime.date(2016, 8, 10), datetime.date(2016, 8, 11),
       datetime.date(2016, 8, 12), datetime.date(2016, 8, 15),
     

In [25]:
import numpy as np

In [35]:
df = get_price('000001.XSHE',end_date='2019-07-12',count=21,fields=['close'])['close']
far = df.iloc[-1,]/df.iloc[0,] - 1

In [39]:
def factor_cal(pool,date):
    df = get_price(pool,end_date=date,count=21,fields=['close'])['close'] 
    far = df.iloc[-1,]/df.iloc[0,] - 1
    return far
factor_cal(['600000.XSHG'],'2019-07-12')
factor_cal(['000001.XSHE'],'2019-07-12')

0.14238683127572016

In [20]:
import pandas as pd

In [80]:
#定义一个空的dataframe记录因子值
factor_df = pd.DataFrame()
#循环计算给定日期范围的因子值
mark = 1
for d in date_list:
    pool = get_index_stocks('000905.XSHG',date=d)
    far = pd.DataFrame([factor_cal(pool,d)])
    if mark == 1:
        factor_df = far
        mark = 0
    else:
        factor_df = factor_df.append(far)
#将columns更改为可以日期标签
factor_df.T.columns = date_list
factor_df.head(3)

KeyboardInterrupt: 

In [55]:
#数据清洗、包括去极值、标准化、中性化等,并加入y值
for date in date_list:
    #对数据进行处理、标准化、去极值、中性化
    #factor_df = winsorize_med(factor_df, scale=3, inclusive=True, inf2nan=True, axis=0) #中位数去极值处理
    se = standardlize(factor_df[date], inf2nan=True) #对每列做标准化处理
    se = neutralize(se, how=['liquidity'], date=date)#剔除原始因子值与流动性相关的部分
    factor_df[date] = se
#进行转置，调整为分析可用的格式
factor_df = factor_df.T
factor_df.head()

0.4960998439937596

In [72]:
#使用获取的因子值进行单因子分析
far = analyze_factor(factor=factor_df, start_date=date_list[0], end_date=date_list[-1], weight_method='avg', industry='jq_l1', quantiles=quantiles, periods=periods,max_loss=0.3)

已退出


In [73]:
# 打印信息比率（IC）相关表
far.plot_information_table(group_adjust=False, method='rank')

auth success 


In [ ]:
# 画各分位数平均收益图
far.plot_quantile_returns_bar(by_group=False, demeaned=0, group_adjust=False)

In [ ]:
# 打印换手率表
far.plot_turnover_table()

In [ ]:
#调用因子分析方法，进行因子信息全览
far.create_full_tear_sheet(demeaned=False, group_adjust=False, by_group=False, turnover_periods=None, avgretplot=(5, 15), std_bar=False)